In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from preprocessing.preprocess_ucr import UCRDatasetImporter
from preprocessing.preprocess_ucr import UCRDataset
from preprocessing.preprocess_ucr import AugUCRDataset
from preprocessing.data_pipeline import build_data_pipeline
from utils import (
    load_yaml_param_settings,
    get_root_dir,
    model_filename,
)
from examine_data import Examiner
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'preprocessing'

In [ ]:
# ['SonyAIBORobotSurface1', 'SonyAIBORobotSurface2', 'Symbols','Mallat']
UCR_SUBSET = [
    "ElectricDevices",
    "StarLightCurves",
    "Wafer",
    "ECG5000",
    "TwoPatterns",
    "FordA",
    "UWaveGestureLibraryAll",
    "FordB",
    "ShapesAll",
    'SonyAIBORobotSurface1', 
    'SonyAIBORobotSurface2', 
    'Symbols',
    'Mallat'
]
examiner = Examiner(datasets=UCR_SUBSET)